# Pathway

lamindb provides access to the following public protein ontologies through [bionty](https://lamin.ai/docs/bionty):

1. [Gene Ontology](https://bioportal.bioontology.org/ontologies/GO)
2. [Pathway Ontology](https://bioportal.bioontology.org/ontologies/PW)

Here we show how to access and search Pathway ontologies to standardize new data.

In [ ]:
import bionty as bt
import pandas as pd

## PublicOntology objects

Let us create a public ontology accessor with `.public` method, which chooses a default public ontology source from {class}`~docs:bionty.PublicSource`.
It's a [PublicOntology](https://lamin.ai/docs/bionty.dev.publicontology) object, which you can think about as a public registry:

In [ ]:
pathways = bt.Pathway.public(organism="all")
pathways

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = pathways.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `public.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = pathways.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.acetyl_coa_assimilation_pathway

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["acetyl-CoA assimilation pathway"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = pathways.lookup(pathways.ontology_id)

In [ ]:
lookup.go_0019681

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
pathways.search("acetyl coa assimilation").head(3)

By default, search also covers synonyms:

In [ ]:
pathways.search("acetyl-CoA catabolism").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
pathways.search("acetyl-CoA catabolism", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
pathways.search(
    "Chemical reactions and pathways resulting in the breakdown of Cinnamic Acid,3-Phenyl-2-Propenoic Acid.",
    field=pathways.definition,
).head()

## Standardize Pathway identifiers

Let us generate a `DataFrame` that stores a number of Pathway identifiers, some of which corrupted:

In [ ]:
df_orig = pd.DataFrame(
    index=[
        "GO:1905210",
        "GO:1905211",
        "GO:1905212",
        "GO:1905208",
        "This pathway does not exist",
    ]
)
df_orig

We can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = pathways.validate(df_orig.index, pathways.name)
df_orig.index[~validated]

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
bt.PublicSource.filter(entity="Pathway").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
public_source = bt.PublicSource.filter(
    source="go", version="2023-05-10", organism="all"
).one()
pathways= bt.Pathway.public(public_source=public_source)
pathways

The currently used ontologies can be displayed using:

In [ ]:
bt.PublicSource.filter(currently_used=True).df()